In [1]:
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
pd.set_option('display.max_columns', 500)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
## read in csv files
data_2013 = pd.read_csv("../data/2012_13_loan_data/LoanStats3b.csv", low_memory = False, encoding='latin-1')

data_a_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_A.csv", low_memory = False);
data_b_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_B.csv", low_memory = False);
data_c_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_C.csv", low_memory = False);
data_d_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_D.csv", low_memory = False);

data_a_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_A.csv", low_memory = False);
data_b_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_B.csv", low_memory = False);
data_c_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_C.csv", low_memory = False);
data_d_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_D.csv", low_memory = False);
data_e_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_E.csv", low_memory = False);
data_f_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_F.csv", low_memory = False);
data_g_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_G.csv", low_memory = False);
data_h_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_H.csv", low_memory = False);


In [3]:
## concatenate data into training set
data_train = pd.concat([data_a_2014, data_b_2014, data_c_2014, data_d_2014, data_2013], ignore_index=True)

data_2015 = pd.concat([data_a_2015, data_b_2015, data_c_2015, data_d_2015, 
                       data_e_2015, data_f_2015, data_g_2015, data_h_2015], 
                       ignore_index=True)

In [4]:
data_train.shape, data_2015.shape

((370443, 145), (421095, 145))

In [5]:
data_train = data_train[pd.notnull(data_train['revol_util'])]
data_2015 = data_2015[pd.notnull(data_2015['revol_util'])]

In [6]:
data_train.shape, data_2015.shape

((370240, 145), (420933, 145))

In [ ]:
## Percent of loans that were fully paid but had non-zero delinquencies in the past 2 years
# len(data_train[(data_train.delinq_2yrs > 0) & (data_train.loan_status == 'Fully Paid')])/float(len(data_train))

In [28]:
# data_train.iloc[86645,:]

In [5]:
# plt.figure(figsize = (20, 10))
# sns.boxplot(y="delinq_2yrs", x="grade", data=data_train);

In [7]:
## checking for policy code 2 in train and test
assert sum(data_train.policy_code == 2) == 0
assert sum(data_2015.policy_code == 2) == 0

In [9]:

## Helper functions
def convert_dates(x):
    
    # Ignore dates with NaNs - will deal with it when taking difference
    if pd.isnull(x) is False:
        try:
            x = datetime.datetime.strptime(x,'%b-%y')
        except:
            x = datetime.datetime.strptime(x,'%b-%Y')
    return x

def diff_year(x, d1, d2):
    # Only change term for repaid loans
    if x['loan_status'] in ['Fully Paid', 'paid']:
        m = (d1.year - d2.year) + (d1.month - d2.month)/12
        
        # For loans that were fully paid back in the same month that they were issued,
        # assume they were repaid in 15 days
        if m == 0 and x['last_pymnt_amnt'] >= x['funded_amnt']:
            m = 0.04

    # For distressed loans, use original term in years
    else:
        m = 1/12 * x['terms_in_months']
    return m

def get_term_adj(df):
    
    # Apply date format
    df['issue_d1'] = df['issue_d'].apply(lambda x: convert_dates(x))
    df['last_pymnt_d1'] = df['last_pymnt_d'].apply(lambda x: convert_dates(x))
    
    # Get term in months
    df['terms_in_months'] = df['term'].apply(lambda x: int(x[1:3]))
    
    # Apply difference
    df['term_adj'] = df.apply(lambda x: diff_year(x, x['last_pymnt_d1'], x['issue_d1']), axis=1)
    
    # Drop helper cols
    df.drop(['terms_in_months', 'issue_d1','last_pymnt_d1'], axis=1)
    
    return df

def format_revol(val):
    percent = val[:-1]
    return float(percent)/100

# def format_interest(val):
#     percent = val[:-1]
#     return float(percent)/100

# def format_term(val):
#     term = val[1:3]
#     return float(term)/12

In [10]:
def initial_model(df, columns, columns_to_encode, columns_to_normalize):
    min_max_scaler = preprocessing.MinMaxScaler()
    new_df = df.copy()
    new_df = new_df[columns]
    new_df = pd.get_dummies(new_df, columns = columns_to_encode)
    new_df['amnt'] = new_df['funded_amnt']
    new_df[columns_to_normalize] = min_max_scaler.fit_transform(new_df[columns_to_normalize])
    return new_df

def clean_data(old_df):
    
    # Start with a fresh copy of the df
    df = old_df.copy()
    
    # Get adjusted term
    df = get_term_adj(df)
    
    # Drop cols with redacted identifying info
    columns_to_drop = ['id', 'member_id', 'emp_title']
    df = df.drop(columns_to_drop, axis = 1)
    
    # Cols to keep
    cols = ['funded_amnt', 'emp_length', 'home_ownership', 'int_rate', 'purpose', 'total_pymnt',
        'annual_inc', 'verification_status', 'dti', 'loan_status', 'revol_util', 'grade', 
            'term', 'term_adj', 'zip_code', 'delinq_2yrs']
    # Cols to one hot encode
    cols_encode = ['emp_length', 'home_ownership', 'verification_status', 'grade', 'purpose']
    # Cols to normalize
    cols_normalize = ['funded_amnt', 'annual_inc', 'dti']
    
    ## Apply initial model fn to clean the df
    df = initial_model(df, cols, cols_encode, cols_normalize)

    # Format using helper fns
    df['term'] = df['term'].apply(lambda x: int(x[1:3]))
    
#     print(set(df['int_rate']))
    df['int_rate'] = df['int_rate'].apply(lambda x: float(x[:-1])/100)

#     df['revol_util'] = df['revol_util'].apply(lambda x: float(x[:-1])/100)
    
    df['revol_util'] = df['revol_util'].map(format_revol)
#     print(set(df['int_rate']))
    
    
    # Mean imputation for revolving utilization and interest rate
#     df[df['revol_util'].isnull()] = np.mean(df['revol_util'])
#     df[df['int_rate'].isnull()] = np.mean(df['int_rate'])
#     print(set(df['int_rate']))
        
    # Delete any current loans
    df = df[df.loan_status != 'Current']
    print(set(df['int_rate']))
        
    # Reduce loan status to a flag <-- RESPONSE VAR
    df['paid'] = 1;
    df.loc[df.loan_status == 'Fully Paid', 'paid'] = 0;
            
    return df

In [11]:
data_train_ = clean_data(data_train)
# df_2015 = clean_data(data_2015)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


{0.1431, 0.21989999999999998, 0.2299, 0.1714, 0.20989999999999998, 0.258, 0.0867, 0.2583, 0.1531, 0.2118, 0.1997, 0.14980000000000002, 0.162, 0.1875, 0.1305, 0.10490000000000001, 0.0949, 0.12119999999999999, 0.22469999999999998, 0.1367, 0.1409, 0.2198, 0.1757, 0.1999, 0.20199999999999999, 0.18989999999999999, 0.1353, 0.1776, 0.21, 0.222, 0.1433, 0.1777, 0.07490000000000001, 0.0649, 0.07690000000000001, 0.08900000000000001, 0.079, 0.0971, 0.1016, 0.20489999999999997, 0.237, 0.2115, 0.1335, 0.19219999999999998, 0.2343, 0.1659, 0.1416, 0.2606, 0.1699, 0.192, 0.1678, 0.21600000000000003, 0.1235, 0.18489999999999998, 0.23829999999999998, 0.2376, 0.0712, 0.1155, 0.1855, 0.2245, 0.1561, 0.2408, 0.1559, 0.1499, 0.15990000000000001, 0.1924, 0.1015, 0.18239999999999998, 0.1398, 0.1285, 0.1522, 0.2528, 0.1972, 0.22399999999999998, 0.20800000000000002, 0.1905, 0.21489999999999998, 0.25989999999999996, 0.12390000000000001, 0.17859999999999998, 0.1199, 0.1624, 0.1099, 0.0917, 0.12990000000000002, 0.

In [22]:
data_train_[data_train_.term_adj == 0]

,funded_amnt,int_rate,total_pymnt,annual_inc,dti,loan_status,revol_util,term,term_adj,zip_code,delinq_2yrs,emp_length_1 year,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,amnt,paid


In [32]:
# data_train[data_train.int_rate == 0.5673913164433951]

,funded_amnt,int_rate,total_pymnt,annual_inc,dti,loan_status,revol_util,term,term_adj,zip_code,delinq_2yrs,emp_length_1 year,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,amnt,paid
1133,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,1
3445,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,1
4166,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,1
5656,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,1
7168,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,1
7278,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,1
8609,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0.567391,0

In [87]:
df_2015.columns == data_train.columns

ValueError: Lengths must match to compare

In [54]:
data_train.to_csv('cleaned_2013_14_upd', index = False)
df_2015.to_csv('cleaned_2015_upd', index = False)